# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    words_set = set(text)
    vocab_to_int = {}
    int_to_vocab = {}
    word_number = 1
    for word in words_set:
        vocab_to_int[word] = word_number
        int_to_vocab[word_number] = word
        word_number = word_number + 1
        
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [20]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """    
    tokens_dict = {'.': 'Period', 
                   ',': 'Comma',                    
                   ';': 'Semicolon',
                   '"': 'Quotation_Mark',
                   '!': 'Exclamation_mark',
                   '?': 'Question_Mark',
                   '(': 'Left_Parentheses',
                   ')': 'Right_Parentheses',
                   '--': 'Dash',
                   '\n': 'Return'}   
  
    return tokens_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/home/george/dlearning/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """    
    Input = tf.placeholder(tf.int32, name="input", shape = [None,None])
    Targets = tf.placeholder(tf.int32, shape = [None,None])
    Learning_rate = tf.placeholder(tf.float32)
    return Input, Targets, Learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [22]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """    
    lstm_layers = 2
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_layers, state_is_tuple=True)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * rnn_size) 
    state = tf.identity(cell.zero_state(batch_size, tf.float32),name='initial_state')    
    return cell, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)
#get_init_cell(10,5)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [23]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1)) 
    embed = tf.nn.embedding_lookup(embedding,input_data)
    return embed

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [24]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """    
    rnn_outputs,final_state = tf.nn.dynamic_rnn(cell,inputs, dtype=tf.float32)
    final_state = tf.identity(final_state,name="final_state")
    
    return rnn_outputs, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [25]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """    
    embed = get_embed(input_data,vocab_size,embed_dim)    
    outputs, final_state = build_rnn(cell,embed)    
    logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=None)
    return logits, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [26]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    total_words = len(int_text)
    total_batches = total_words // (batch_size*seq_length)
    
    inputs_start = 0;
    inputs_end = total_batches*batch_size*seq_length
    inputs_array = np.array(int_text[inputs_start:inputs_end])
    inputs = np.split(np.array(int_text[inputs_start:inputs_end]).reshape(batch_size, -1), total_batches, 1)
    
    targets_start = inputs_start + 1
    targets_end = inputs_end + 1
    targets = np.split(np.array(int_text[targets_start:targets_end]).reshape(batch_size, -1), total_batches, 1)
   
    # set the last value of targets to the first value in inputs
    targets[total_batches-1][batch_size-1][seq_length-1] = inputs[0][0][0]    
    
    final_batch = np.array(list(zip(inputs, targets))).reshape(total_batches, 2, batch_size, seq_length)

    return final_batch

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)

tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [42]:
# Number of Epochs
num_epochs = 10
# Batch Size
batch_size = 3
# RNN Size
rnn_size = 1
# Embedding Dimension Size
embed_dim = 1
# Sequence Length
seq_length = 2
# Learning Rate
learning_rate = 0.1
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [43]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/11516   train_loss = 8.822
Epoch   0 Batch    1/11516   train_loss = 8.822
Epoch   0 Batch    2/11516   train_loss = 8.821
Epoch   0 Batch    3/11516   train_loss = 8.823
Epoch   0 Batch    4/11516   train_loss = 8.558
Epoch   0 Batch    5/11516   train_loss = 8.574
Epoch   0 Batch    6/11516   train_loss = 8.581
Epoch   0 Batch    7/11516   train_loss = 8.420
Epoch   0 Batch    8/11516   train_loss = 8.061
Epoch   0 Batch    9/11516   train_loss = 8.210
Epoch   0 Batch   10/11516   train_loss = 8.247
Epoch   0 Batch   11/11516   train_loss = 7.920
Epoch   0 Batch   12/11516   train_loss = 7.358
Epoch   0 Batch   13/11516   train_loss = 7.195
Epoch   0 Batch   14/11516   train_loss = 6.722
Epoch   0 Batch   15/11516   train_loss = 6.834
Epoch   0 Batch   16/11516   train_loss = 7.942
Epoch   0 Batch   17/11516   train_loss = 6.980
Epoch   0 Batch   18/11516   train_loss = 5.954
Epoch   0 Batch   19/11516   train_loss = 6.512
Epoch   0 Batch   20/11516   train_loss 

Epoch   0 Batch  193/11516   train_loss = 5.831
Epoch   0 Batch  194/11516   train_loss = 8.244
Epoch   0 Batch  195/11516   train_loss = 6.326
Epoch   0 Batch  196/11516   train_loss = 5.074
Epoch   0 Batch  197/11516   train_loss = 5.409
Epoch   0 Batch  198/11516   train_loss = 6.831
Epoch   0 Batch  199/11516   train_loss = 6.211
Epoch   0 Batch  200/11516   train_loss = 8.020
Epoch   0 Batch  201/11516   train_loss = 9.134
Epoch   0 Batch  202/11516   train_loss = 4.677
Epoch   0 Batch  203/11516   train_loss = 6.909
Epoch   0 Batch  204/11516   train_loss = 5.952
Epoch   0 Batch  205/11516   train_loss = 4.828
Epoch   0 Batch  206/11516   train_loss = 6.724
Epoch   0 Batch  207/11516   train_loss = 6.960
Epoch   0 Batch  208/11516   train_loss = 6.073
Epoch   0 Batch  209/11516   train_loss = 7.712
Epoch   0 Batch  210/11516   train_loss = 7.538
Epoch   0 Batch  211/11516   train_loss = 7.103
Epoch   0 Batch  212/11516   train_loss = 4.755
Epoch   0 Batch  213/11516   train_loss 

Epoch   0 Batch  381/11516   train_loss = 6.134
Epoch   0 Batch  382/11516   train_loss = 8.970
Epoch   0 Batch  383/11516   train_loss = 5.627
Epoch   0 Batch  384/11516   train_loss = 6.692
Epoch   0 Batch  385/11516   train_loss = 5.498
Epoch   0 Batch  386/11516   train_loss = 6.605
Epoch   0 Batch  387/11516   train_loss = 7.156
Epoch   0 Batch  388/11516   train_loss = 6.157
Epoch   0 Batch  389/11516   train_loss = 5.618
Epoch   0 Batch  390/11516   train_loss = 8.498
Epoch   0 Batch  391/11516   train_loss = 8.806
Epoch   0 Batch  392/11516   train_loss = 7.618
Epoch   0 Batch  393/11516   train_loss = 5.113
Epoch   0 Batch  394/11516   train_loss = 5.163
Epoch   0 Batch  395/11516   train_loss = 8.953
Epoch   0 Batch  396/11516   train_loss = 5.367
Epoch   0 Batch  397/11516   train_loss = 7.545
Epoch   0 Batch  398/11516   train_loss = 7.270
Epoch   0 Batch  399/11516   train_loss = 5.377
Epoch   0 Batch  400/11516   train_loss = 6.568
Epoch   0 Batch  401/11516   train_loss 

Epoch   0 Batch  572/11516   train_loss = 7.142
Epoch   0 Batch  573/11516   train_loss = 6.076
Epoch   0 Batch  574/11516   train_loss = 7.104
Epoch   0 Batch  575/11516   train_loss = 7.213
Epoch   0 Batch  576/11516   train_loss = 6.510
Epoch   0 Batch  577/11516   train_loss = 7.257
Epoch   0 Batch  578/11516   train_loss = 7.281
Epoch   0 Batch  579/11516   train_loss = 6.242
Epoch   0 Batch  580/11516   train_loss = 3.695
Epoch   0 Batch  581/11516   train_loss = 6.168
Epoch   0 Batch  582/11516   train_loss = 8.793
Epoch   0 Batch  583/11516   train_loss = 6.660
Epoch   0 Batch  584/11516   train_loss = 3.313
Epoch   0 Batch  585/11516   train_loss = 6.348
Epoch   0 Batch  586/11516   train_loss = 5.691
Epoch   0 Batch  587/11516   train_loss = 6.569
Epoch   0 Batch  588/11516   train_loss = 9.988
Epoch   0 Batch  589/11516   train_loss = 6.012
Epoch   0 Batch  590/11516   train_loss = 5.717
Epoch   0 Batch  591/11516   train_loss = 4.761
Epoch   0 Batch  592/11516   train_loss 

Epoch   0 Batch  760/11516   train_loss = 6.096
Epoch   0 Batch  761/11516   train_loss = 2.863
Epoch   0 Batch  762/11516   train_loss = 4.674
Epoch   0 Batch  763/11516   train_loss = 5.107
Epoch   0 Batch  764/11516   train_loss = 5.753
Epoch   0 Batch  765/11516   train_loss = 7.025
Epoch   0 Batch  766/11516   train_loss = 7.338
Epoch   0 Batch  767/11516   train_loss = 8.671
Epoch   0 Batch  768/11516   train_loss = 4.186
Epoch   0 Batch  769/11516   train_loss = 6.330
Epoch   0 Batch  770/11516   train_loss = 5.725
Epoch   0 Batch  771/11516   train_loss = 7.190
Epoch   0 Batch  772/11516   train_loss = 8.593
Epoch   0 Batch  773/11516   train_loss = 6.148
Epoch   0 Batch  774/11516   train_loss = 3.927
Epoch   0 Batch  775/11516   train_loss = 6.432
Epoch   0 Batch  776/11516   train_loss = 4.221
Epoch   0 Batch  777/11516   train_loss = 7.113
Epoch   0 Batch  778/11516   train_loss = 8.017
Epoch   0 Batch  779/11516   train_loss = 6.905
Epoch   0 Batch  780/11516   train_loss 

...

Epoch   0 Batch 1538/11516   train_loss = 3.977
Epoch   0 Batch 1539/11516   train_loss = 5.816
Epoch   0 Batch 1540/11516   train_loss = 5.967
Epoch   0 Batch 1541/11516   train_loss = 5.341
Epoch   0 Batch 1542/11516   train_loss = 7.061
Epoch   0 Batch 1543/11516   train_loss = 7.422
Epoch   0 Batch 1544/11516   train_loss = 5.267
Epoch   0 Batch 1545/11516   train_loss = 3.904
Epoch   0 Batch 1546/11516   train_loss = 4.029
Epoch   0 Batch 1547/11516   train_loss = 5.738
Epoch   0 Batch 1548/11516   train_loss = 7.790
Epoch   0 Batch 1549/11516   train_loss = 8.086
Epoch   0 Batch 1550/11516   train_loss = 4.969
Epoch   0 Batch 1551/11516   train_loss = 3.958
Epoch   0 Batch 1552/11516   train_loss = 6.108
Epoch   0 Batch 1553/11516   train_loss = 5.107
Epoch   0 Batch 1554/11516   train_loss = 9.574
Epoch   0 Batch 1555/11516   train_loss = 7.456
Epoch   0 Batch 1556/11516   train_loss = 6.313
Epoch   0 Batch 1557/11516   train_loss = 9.852
Epoch   0 Batch 1558/11516   train_loss 

Epoch   0 Batch 1732/11516   train_loss = 7.226
Epoch   0 Batch 1733/11516   train_loss = 4.914
Epoch   0 Batch 1734/11516   train_loss = 8.764
Epoch   0 Batch 1735/11516   train_loss = 8.922
Epoch   0 Batch 1736/11516   train_loss = 6.172
Epoch   0 Batch 1737/11516   train_loss = 6.547
Epoch   0 Batch 1738/11516   train_loss = 6.086
Epoch   0 Batch 1739/11516   train_loss = 4.290
Epoch   0 Batch 1740/11516   train_loss = 5.721
Epoch   0 Batch 1741/11516   train_loss = 3.860
Epoch   0 Batch 1742/11516   train_loss = 8.433
Epoch   0 Batch 1743/11516   train_loss = 6.897
Epoch   0 Batch 1744/11516   train_loss = 4.457
Epoch   0 Batch 1745/11516   train_loss = 9.861
Epoch   0 Batch 1746/11516   train_loss = 8.178
Epoch   0 Batch 1747/11516   train_loss = 3.605
Epoch   0 Batch 1748/11516   train_loss = 5.882
Epoch   0 Batch 1749/11516   train_loss = 7.799
Epoch   0 Batch 1750/11516   train_loss = 7.577
Epoch   0 Batch 1751/11516   train_loss = 10.025
Epoch   0 Batch 1752/11516   train_loss

Epoch   0 Batch 1919/11516   train_loss = 4.777
Epoch   0 Batch 1920/11516   train_loss = 7.314
Epoch   0 Batch 1921/11516   train_loss = 6.679
Epoch   0 Batch 1922/11516   train_loss = 3.245
Epoch   0 Batch 1923/11516   train_loss = 6.751
Epoch   0 Batch 1924/11516   train_loss = 6.657
Epoch   0 Batch 1925/11516   train_loss = 4.295
Epoch   0 Batch 1926/11516   train_loss = 4.725
Epoch   0 Batch 1927/11516   train_loss = 7.649
Epoch   0 Batch 1928/11516   train_loss = 5.948
Epoch   0 Batch 1929/11516   train_loss = 6.904
Epoch   0 Batch 1930/11516   train_loss = 4.709
Epoch   0 Batch 1931/11516   train_loss = 6.952
Epoch   0 Batch 1932/11516   train_loss = 8.214
Epoch   0 Batch 1933/11516   train_loss = 10.020
Epoch   0 Batch 1934/11516   train_loss = 4.748
Epoch   0 Batch 1935/11516   train_loss = 4.734
Epoch   0 Batch 1936/11516   train_loss = 5.048
Epoch   0 Batch 1937/11516   train_loss = 6.635
Epoch   0 Batch 1938/11516   train_loss = 6.250
Epoch   0 Batch 1939/11516   train_loss

Epoch   0 Batch 2115/11516   train_loss = 7.811
Epoch   0 Batch 2116/11516   train_loss = 6.703
Epoch   0 Batch 2117/11516   train_loss = 7.557
Epoch   0 Batch 2118/11516   train_loss = 5.232
Epoch   0 Batch 2119/11516   train_loss = 6.754
Epoch   0 Batch 2120/11516   train_loss = 5.417
Epoch   0 Batch 2121/11516   train_loss = 8.995
Epoch   0 Batch 2122/11516   train_loss = 5.214
Epoch   0 Batch 2123/11516   train_loss = 6.645
Epoch   0 Batch 2124/11516   train_loss = 8.917
Epoch   0 Batch 2125/11516   train_loss = 4.804
Epoch   0 Batch 2126/11516   train_loss = 4.797
Epoch   0 Batch 2127/11516   train_loss = 7.706
Epoch   0 Batch 2128/11516   train_loss = 6.315
Epoch   0 Batch 2129/11516   train_loss = 2.963
Epoch   0 Batch 2130/11516   train_loss = 9.153
Epoch   0 Batch 2131/11516   train_loss = 5.022
Epoch   0 Batch 2132/11516   train_loss = 6.727
Epoch   0 Batch 2133/11516   train_loss = 5.339
Epoch   0 Batch 2134/11516   train_loss = 5.406
Epoch   0 Batch 2135/11516   train_loss 

InvalidArgumentError: Received a label value of 6776 which is outside the valid range of [0, 6776).  Label values: 3513 6776 5425 1213 2308 1748
	 [[Node: sequence_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](sequence_loss/Reshape, sequence_loss/Reshape_1)]]

Caused by op 'sequence_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/george/dlearning/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/george/dlearning/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/george

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [ ]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return None, None, None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.